# 🗺️ Procedural Fantasy WorldGen — v9 (Modular YAML)

This version loads **all `.yaml` files** in a folder, deep-merges them (later files override earlier ones),
and then runs the same world generators.

Place your world fragments under a directory (default: `mnt/data/world_mod`).


In [28]:
#!pip install pyyaml openai # Uncomment if needed

In [29]:
import re, random, math, json, os, glob
from copy import deepcopy
from datetime import datetime
from typing import Any, Dict, List, Optional
from functools import reduce
import yaml

# Constants for magic strings
PROCGEN_TABLES = 'procgen.tables'
PROCGEN_GENERATORS = 'procgen.generators'
PROCGEN_WEIGHTS = 'procgen.weights'
PROCGEN_DICE = 'procgen.dice'

# Compiled regex patterns (module-level for performance)
DICE_PAT = re.compile(r"(?:(\d+)[dD](\d+))|([()+\-*/])|(\d+)")
CURLY_RE = re.compile(r"\{\{\s*([a-zA-Z0-9_.-]+)(\|[a-zA-Z]+)?\s*\}\}")

# Global state (will be replaced by context object)
CURRENT_ROOT = None
CURRENT_RNG = None
_TABLE_CACHE = {}

def make_rng(seed: Optional[int]) -> random.Random:
    """Create a random number generator with optional seed."""
    rng = random.Random()
    rng.seed(seed if seed is not None else None)
    return rng

def deep_merge(a: dict, b: dict) -> dict:
    """Deep merge two dictionaries. Optimized to avoid double deep-copying."""
    out = a.copy()  # Shallow copy first
    for k, v in b.items():
        if k in out and isinstance(out[k], dict) and isinstance(v, dict):
            out[k] = deep_merge(out[k], v)
        else:
            out[k] = deepcopy(v)  # Only deep copy leaf values
    return out

def load_yaml_file(path: str) -> dict:
    """Load a single YAML file."""
    with open(path, 'r', encoding='utf-8') as f:
        return yaml.safe_load(f) or {}

def load_world_folder(folder: str) -> dict:
    """Load and merge all YAML files in a folder. Optimized with reduce."""
    files = sorted(glob.glob(os.path.join(folder, '*.yaml')))
    if not files:
        return {}
    fragments = [load_yaml_file(p) for p in files]
    return reduce(deep_merge, fragments, {})

def _roll_single(rng: random.Random, ndice: int, sides: int) -> int:
    """Roll ndice with given number of sides."""
    return sum(rng.randint(1, sides) for _ in range(ndice))

def roll(expr: str, rng: Optional[random.Random] = None) -> int:
    """Parse and evaluate a dice expression like '2d6+3'."""
    if not expr:
        raise ValueError("Empty dice expression")
    if rng is None:
        rng = CURRENT_RNG
    tokens = []
    for m in DICE_PAT.finditer(str(expr)):
        if m.group(1) and m.group(2):
            nd, sd = int(m.group(1)), int(m.group(2))
            tokens.append(str(_roll_single(rng, nd, sd)))
        elif m.group(3):
            tokens.append(m.group(3))
        elif m.group(4):
            tokens.append(m.group(4))
    safe = ''.join(tokens)
    try:
        return int(eval(safe, {"__builtins__": {}}, {}))
    except Exception as e:
        raise ValueError(f"Bad dice expr '{expr}' -> '{safe}': {e}")

def weighted_choice(weights: Dict[str, int], rng: random.Random) -> str:
    """Choose a random key from weights dict. Optimized with random.choices."""
    if not weights:
        raise ValueError("Empty weights dictionary")
    items = list(weights.keys())
    wts = [max(int(w), 0) for w in weights.values()]
    total = sum(wts)
    if total <= 0:
        raise ValueError("Weights sum to zero")
    return rng.choices(items, weights=wts, k=1)[0]

def deep_get(d: Dict[str, Any], path: str) -> Any:
    """Navigate nested dict using dot-separated path."""
    cur = d
    for part in path.split('.'):
        if part == '':
            continue
        if isinstance(cur, dict) and part in cur:
            cur = cur[part]
        else:
            raise KeyError(f"Path '{path}' not found (stuck at '{part}')")
    return cur

def interpolate(s: str, ctx: Dict[str, Any]) -> str:
    """Replace {{key}} and {{key|filter}} patterns with context values."""
    def repl(m):
        key = m.group(1)
        filt = (m.group(2) or '').lstrip('|')
        val = ctx
        for p in key.split('.'):
            if isinstance(val, dict) and p in val:
                val = val[p]
            else:
                return m.group(0)
        sval = str(val)
        if   filt == 'title': sval = sval.title()
        elif filt == 'upper': sval = sval.upper()
        elif filt == 'lower': sval = sval.lower()
        return sval
    return CURLY_RE.sub(repl, s)

def case(key: Any, mapping: Dict[str, Any], default=None):
    """Simple case/switch statement helper."""
    return mapping.get(str(key), default)

def treasure_tier_by_cr(cr_val: Any) -> Any:
    """Look up treasure tier based on CR value."""
    if CURRENT_ROOT is None:
        raise RuntimeError("CURRENT_ROOT not set")
    try:
        cr = float(cr_val)
    except Exception:
        cr = 0.0
    table = deep_get(CURRENT_ROOT, f'{PROCGEN_DICE}.treasure_tier_by_cr')
    
    # Single-pass lookup with priority: exact match > range > plus
    exact_match = None
    range_match = None
    plus_match = None
    
    for k, v in table.items():
        k_str = str(k).strip()
        
        # Check for exact match
        try:
            if float(k_str) == cr:
                exact_match = v
                break
        except ValueError:
            pass
        
        # Check for range (e.g., "3-6")
        if '-' in k_str:
            try:
                a, b = k_str.split('-', 1)
                if float(a) <= cr <= float(b):
                    range_match = v
            except ValueError:
                pass
        
        # Check for plus (e.g., "15+")
        if k_str.endswith('+'):
            try:
                start = float(k_str[:-1])
                if cr >= start:
                    plus_match = v
            except ValueError:
                pass
    
    return exact_match or range_match or plus_match or next(iter(table.values()))

def lookup(path: str) -> Any:
    """Look up a value in the world root by path."""
    if CURRENT_ROOT is None:
        raise RuntimeError("CURRENT_ROOT not set")
    return deep_get(CURRENT_ROOT, path)

class SafeLocals(dict):
    """Safe locals dict for eval that includes context variables."""
    RESERVED = {"treasure_tier_by_cr","lookup","roll","case","math",
                "max","min","abs","round","int","float","len","sum"}
    
    def __init__(self, base_ctx: Dict[str, Any]):
        super().__init__()
        self._ctx = base_ctx
    
    def __contains__(self, key):
        return dict.__contains__(self, key) or (key in self._ctx and key not in self.RESERVED)
    
    def __getitem__(self, key):
        if dict.__contains__(self, key):
            return dict.__getitem__(self, key)
        if key in self._ctx and key not in self.RESERVED:
            return self._ctx[key]
        raise KeyError(key)
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default

def eval_expr(expr: str, ctx: Dict[str, Any]) -> Any:
    """Safely evaluate an expression with context."""
    safe_locals = SafeLocals(ctx)
    safe_locals.update({
        "case": case,
        "math": math,
        "treasure_tier_by_cr": treasure_tier_by_cr,
        "lookup": lookup,
        "roll": lambda s: roll(s, CURRENT_RNG),
        "max": max, "min": min, "abs": abs, "round": round,
        "int": int, "float": float, "len": len, "sum": sum,
    })
    try:
        return eval(expr, {"__builtins__": {}}, safe_locals)
    except Exception as e:
        raise ValueError(f"Bad derive/bind expr '{expr}': {e}")

def resolve_at(ref: str, root: Dict[str, Any], runtime: Dict[str, Any]) -> Any:
    """Resolve @-prefixed references to world data."""
    if not ref.startswith('@'):
        return ref
    path = ref[1:]
    if path.startswith('weights.'):
        path = f'{PROCGEN_WEIGHTS}.{path[8:]}'
    elif path.startswith('dice.'):
        path = f'{PROCGEN_DICE}.{path[5:]}'
    
    # Conditional interpolation - only if needed
    if '{{' in path:
        path = interpolate(path, runtime)
    
    if '[' in path and path.endswith(']'):
        head, key = path.split('[', 1)
        key = key[:-1]
        base = deep_get(root, head)
        return base[key]
    return deep_get(root, path)

def build_table_cache(root: Dict[str, Any]) -> None:
    """Build lookup cache for procgen tables."""
    global _TABLE_CACHE
    try:
        tables = deep_get(root, PROCGEN_TABLES)
        _TABLE_CACHE = {t['id']: t for t in tables if 'id' in t}
    except KeyError:
        _TABLE_CACHE = {}

def get_proc_table(root: Dict[str, Any], table_id: str) -> Dict[str, Any]:
    """Get a procgen table by ID. Uses cache for performance."""
    if not _TABLE_CACHE:
        build_table_cache(root)
    if table_id not in _TABLE_CACHE:
        raise KeyError(f"Table '{table_id}' not found in {PROCGEN_TABLES}")
    return _TABLE_CACHE[table_id]

def roll_table(table: Dict[str, Any], rng: random.Random) -> Any:
    """Roll on a weighted table and return a result."""
    entries = table.get('entries', [])
    if not entries:
        return None
    total = sum(int(e.get('weight', 1)) for e in entries)
    r = rng.uniform(0, total)
    acc = 0
    for e in entries:
        acc += int(e.get('weight', 1))
        if r <= acc:
            return e.get('result', e)
    return entries[-1].get('result', entries[-1])

def run_generator(root: Dict[str, Any], gen_name: str, rng: random.Random, 
                  base_ctx: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """Run a named generator and return the resulting context."""
    gens = deep_get(root, PROCGEN_GENERATORS)
    if gen_name not in gens:
        raise KeyError(f"Generator '{gen_name}' not found in {PROCGEN_GENERATORS}")
    steps = gens[gen_name].get('steps', [])
    ctx = {} if base_ctx is None else deepcopy(base_ctx)
    ctx['now'] = datetime.utcnow().isoformat()

    for step in steps:
        if 'choose' in step:
            spec = step['choose']
            for k, v in spec.items():
                if isinstance(v, str) and v.startswith('@'):
                    resolved = resolve_at(v, root, ctx)
                    if isinstance(resolved, dict):
                        ctx[k] = weighted_choice(resolved, rng)
                    else:
                        ctx[k] = resolved
                else:
                    ctx[k] = v

        elif 'roll' in step:
            spec = step['roll']
            for k, v in spec.items():
                if isinstance(v, str) and v.startswith('@'):
                    resolved = resolve_at(v, root, ctx)
                    expr = resolved
                    ctx[k] = roll(str(expr), rng)
                else:
                    ctx[k] = roll(str(v), rng)

        elif 'pick_n' in step:
            spec = step['pick_n']
            src = spec.get('from', [])
            n_expr = spec.get('n', 1)
            n = roll(str(n_expr), rng) if isinstance(n_expr, str) else int(n_expr)
            pool = list(src)
            rng.shuffle(pool)
            ctx['problems'] = pool[:n]

        elif 'derive' in step:
            spec = step['derive']
            for k, expr in spec.items():
                ctx[k] = eval_expr(str(expr), ctx)

        elif 'table' in step:
            table_id = step['table']
            t = get_proc_table(root, table_id)
            res = roll_table(t, rng)
            ctx[table_id] = res
            alias = table_id.split('_')[0]
            if alias and alias not in ctx:
                ctx[alias] = res

        elif 'bind' in step:
            spec = step['bind']
            for k, expr in spec.items():
                ctx[k] = eval_expr(str(expr), ctx) if isinstance(expr, str) else expr

        elif 'ensure' in step:
            pass

        elif 'output_template' in step:
            tmpl = step['output_template']
            ctx['output'] = interpolate(tmpl, ctx)

    return ctx

def list_generators(root: Dict[str, Any]) -> List[str]:
    """List all available generator names."""
    gens = deep_get(root, PROCGEN_GENERATORS)
    return sorted(gens.keys())

In [30]:
from pprint import pprint

WORLD_DIR = 'mnt/data/world_mod'  # change if needed
root = load_world_folder(WORLD_DIR)

seed = (root.get('meta', {}) or {}).get('seed')
rng = make_rng(seed)
CURRENT_ROOT = root
CURRENT_RNG = rng

print('Loaded folder:', WORLD_DIR)
print('Merged keys:', list(root.keys()))
print('Seed:', seed)
print('Generators:', ', '.join(list_generators(root)))

Loaded folder: mnt/data/world_mod
Merged keys: ['meta', 'tone', 'cosmology', 'calendar', 'geography', 'cultures', 'species', 'factions', 'settlements', 'dungeons', 'npcs', 'bestiary', 'names', 'history', 'procgen', 'encounters', 'loot', 'house_rules']
Seed: 123456
Generators: dungeon, quest, settlement


In [31]:
# Run sample generators and save outputs
samples_to_run = ['settlement', 'dungeon', 'quest']
outputs = {}
for gname in samples_to_run:
    try:
        outputs[gname] = run_generator(root, gname, rng)
    except KeyError as e:
        outputs[gname] = {"error": str(e)}

from pprint import pprint
pprint(outputs)

for gname, data in outputs.items():
    out = data.get('output') if isinstance(data, dict) else None
    if out:
        print(f"\n--- {gname.upper()} TEMPLATE OUTPUT ---\n{out}")

save_path = 'mnt/data/worldgen_outputs.json'
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(outputs, f, ensure_ascii=False, indent=2)
print('\nSaved JSON:', save_path)

quest_out = outputs.get('quest', {}).get('output') if isinstance(outputs.get('quest'), dict) else None
if quest_out:
    qpath = 'mnt/data/worldgen_quest.txt'
    with open(qpath, 'w', encoding='utf-8') as f:
        f.write(quest_out)
    print('Saved quest text:', qpath)

{'dungeon': {'cr_guess': 2,
             'depth': 3,
             'now': '2025-11-11T20:31:46.971496',
             'region': 'ash_desert',
             'theme': 'ruin_arcane'},
 'quest': {'complication': 'A rival faction interferes.',
           'cr_guess': 3,
           'difficulty_baseline': 'standard',
           'hook': {'twist': 'is_the_villain', 'type': 'missing_person'},
           'hook_types': {'twist': 'is_the_villain', 'type': 'missing_person'},
           'now': '2025-11-11T20:31:46.971525',
           'output': 'Missing_Person in Riftmarch: is_the_villain.\n'
                     'Complication: A rival faction interferes.\n'
                     'Reward Tier: 1d6.\n',
           'region': 'riftmarch',
           'reward': '1d6'},
 'settlement': {'biome': 'coast',
                'defense_rating': 1,
                'market_size': 'tiny',
                'now': '2025-11-11T20:31:46.971403',
                'population': 90,
                'problems': [],
                '

# 🚀 Adventure Export & OpenAI Integration

This section provides **Markdown adventure exports** with optional **OpenAI API** enhancement.

## Features

- **Local Markdown Export**: Pure Python templating, no API required
- **OpenAI Enhancement**: Optional AI-powered narrative expansion and formatting
- **Flexible Schema**: Adapts to various generator output formats

## Quick Start

1. Run your world generator to create adventure data (e.g., `outputs` dict)
2. Use `save_markdown()` for instant local export
3. (Optional) Use `call_openai_and_save()` for AI-enhanced version

## Expected Schema

The exporter expects a dict with this structure:

```python
adventure_dict = {
  "adventure": {
    "title": "The Howling Crypt of Riftmarch",
    "region": "Riftmarch",
    "theme": "tomb_cursed",
    "quest_summary": {
      "type": "monster_hunt",
      "description": "...",
      "complication": "rival faction",
      "reward": "1d6"
    },
    "hooks": [{"title": "...", "description": "..."}],
    "locations": {
      "settlement": {"name": "...", "type": "city", "population": 1234},
      "dungeon": {"name": "...", "depth": 4, "cr_guess": 3}
    },
    "monster_encounters": [{"name": "...", "type": "...", "cr": 3, "abilities": ["..."]}],
    "random_encounters": [{"roll": "1-2", "encounter": "..."}],
    "treasure": [{"roll": "1", "treasure": "..."}],
    "procedural_traps": [{"roll": "1", "trap": "...", "description": "..."}],
    "procedural_relics": [{"roll": "1", "relic": "...", "power": "..."}],
    "procedural_dialogue": [{"roll": "1", "npc": "...", "line": "..."}],
    "procedural_events": [{"roll": "1", "event": "..."}],
    "resolution": {"choices": ["...", "..."], "outcome": "..."}
  }
}
```

Use `map_to_standard_schema()` if your generator outputs a different structure.

In [32]:
# =========================
# Local Markdown Exporter
# =========================
from textwrap import dedent
from datetime import datetime
import os, json

EXPORT_DIR = os.environ.get("ADVENTURE_EXPORT_DIR", "mnt/data/exports")
os.makedirs(EXPORT_DIR, exist_ok=True)

def _get(d, path, default=""):
    """Safe getter: path like 'adventure.quest_summary.type'"""
    cur = d
    for p in path.split("."):
        if cur is None:
            return default
        if isinstance(cur, dict):
            cur = cur.get(p, default)
        else:
            return default
    return cur if cur is not None else default

def _as_table(rows, headers=None):
    """Convert list of dicts to markdown table"""
    if not rows:
        return ""
    if headers is None:
        headers = list(rows[0].keys())
    line_sep = "|" + "|".join(["---"] * len(headers)) + "|"
    head = "|" + "|".join(str(h).title().replace("_", " ") for h in headers) + "|"
    body = "\n".join("|" + "|".join(str(r.get(h, "")) for h in headers) + "|" for r in rows)
    return "\n".join([head, line_sep, body])

def render_markdown(adventure_dict: dict) -> str:
    """
    Render adventure dict to clean, game-ready Markdown.
    Handles nested adventure structure and provides sensible defaults.
    """
    adv = adventure_dict.get("adventure", adventure_dict)
    
    # Extract metadata
    title = _get(adventure_dict, "adventure.title", "Untitled Adventure")
    region = _get(adventure_dict, "adventure.region", _get(adventure_dict, "region", ""))
    theme = _get(adventure_dict, "adventure.theme", _get(adventure_dict, "theme", ""))
    qtype = _get(adventure_dict, "adventure.quest_summary.type", _get(adventure_dict, "quest.type", ""))
    qdesc = _get(adventure_dict, "adventure.quest_summary.description", "")
    qcomp = _get(adventure_dict, "adventure.quest_summary.complication", _get(adventure_dict, "quest.complication", ""))
    qreward = _get(adventure_dict, "adventure.quest_summary.reward", _get(adventure_dict, "quest.reward", ""))

    hooks = adv.get("hooks", [])
    locs = adv.get("locations", {})
    settlement = locs.get("settlement", {})
    dungeon = locs.get("dungeon", {})
    monsters = adv.get("monster_encounters", [])
    randoms = adv.get("random_encounters", [])
    resolution = adv.get("resolution", {})
    treasure = adv.get("treasure", [])
    traps = adv.get("procedural_traps", [])
    relics = adv.get("procedural_relics", [])
    npc_dialogue = adv.get("procedural_dialogue", [])
    dyn_events = adv.get("procedural_events", [])

    def stat_block(m):
        """Format monster stat block"""
        lines = [f"### **{m.get('name', 'Unknown')}**"]
        lines.append(f"**Type:** {m.get('type', '')}  ")
        if "cr" in m:
            lines.append(f"**CR:** {m.get('cr')}  ")
        
        # Core stats
        for key in ["alignment", "armor_class", "hit_points", "speed"]:
            val = m.get(key)
            if val:
                lab = key.replace("_", " ").title()
                lines.append(f"**{lab}:** {val}  ")
        
        # Abilities
        abilities = m.get("abilities", [])
        if abilities:
            lines.append("\n**Abilities:**")
            for a in abilities:
                lines.append(f"- {a}")
        
        # Behavior
        behavior = m.get("behavior")
        if behavior:
            lines.append(f"\n**Behavior:** {behavior}")
        
        return "\n".join(lines)

    def _render_section(block, title, default_headers=None):
        """Render a section with optional table formatting"""
        if not block:
            return []
        lines = [f"\n## {title}"]
        if isinstance(block, list) and block and isinstance(block[0], dict):
            headers = default_headers or list(block[0].keys())
            lines.append(_as_table(block, headers=headers))
        else:
            for item in block:
                if isinstance(item, dict):
                    lines.append("- " + ", ".join(f"{k}: {v}" for k, v in item.items()))
                else:
                    lines.append(f"- {item}")
        return lines

    # Build markdown document
    md = [f"# 🗺️ {title}"]
    
    # Metadata
    meta = []
    if region:
        meta.append(f"**Region:** {region}")
    if theme:
        meta.append(f"**Theme:** {theme}")
    if qtype:
        meta.append(f"**Type:** {str(qtype).replace('_', ' ').title()}")
    if qcomp:
        meta.append(f"**Complication:** {qcomp}")
    if qreward:
        meta.append(f"**Reward:** {qreward}")
    if meta:
        md.append("\n" + "  \n".join(meta) + "\n")

    # Summary
    if qdesc:
        md.append("## 📜 Adventure Summary")
        md.append(qdesc)

    # Hooks
    if hooks:
        md.append("\n## ⚔️ Adventure Hooks")
        for i, h in enumerate(hooks, 1):
            t = h.get("title", f"Hook {i}")
            d = h.get("description", "")
            md.append(f"{i}. **{t}** – {d}")

    # Settlement
    if settlement:
        md.append("\n## 🏙️ Settlement")
        for k in ["name", "type", "population", "defense_rating", "market_size", "description"]:
            v = settlement.get(k)
            if v is not None:
                lab = k.replace("_", " ").title()
                md.append(f"**{lab}:** {v}  ")

    # Dungeon
    if dungeon:
        md.append("\n## 🏚️ Dungeon")
        for k in ["name", "depth", "cr_guess", "cr_range", "description"]:
            v = dungeon.get(k)
            if v is not None:
                lab = k.replace("_", " ").title()
                md.append(f"**{lab}:** {v}  ")

    # Monsters
    if monsters:
        md.append("\n## 👹 Monster Encounters")
        for m in monsters:
            md.append(stat_block(m))

    # Random encounters
    if randoms:
        md.append("\n## 🎲 Random Encounters")
        headers = ["roll", "encounter"]
        if all(isinstance(x, dict) and set(x.keys()) >= set(headers) for x in randoms):
            md.append(_as_table(randoms, headers=headers))
        else:
            for r in randoms:
                if isinstance(r, dict):
                    md.append(f"- **{r.get('roll', '?')}**: {r.get('encounter', '')}")
                else:
                    md.append(f"- {r}")

    # Treasure
    if treasure:
        md.extend(_render_section(treasure, "💰 Treasure", default_headers=["roll", "treasure"]))

    # Procedural tables
    if traps:
        md.extend(_render_section(traps, "🪤 Traps", default_headers=["roll", "trap", "description"]))
    if relics:
        md.extend(_render_section(relics, "✨ Relics", default_headers=["roll", "relic", "power"]))
    if npc_dialogue:
        md.extend(_render_section(npc_dialogue, "💬 NPC Dialogue", default_headers=["roll", "npc", "line"]))
    if dyn_events:
        md.extend(_render_section(dyn_events, "⚡ Dynamic Events", default_headers=["roll", "event"]))

    # Resolution
    if resolution:
        md.append("\n## 🎯 Resolution Paths")
        choices = resolution.get("choices", [])
        outcome = resolution.get("outcome", "")
        for c in choices:
            md.append(f"- {c}")
        if outcome:
            md.append(f"\n**Outcome:** {outcome}")

    return "\n\n".join(md)


def save_markdown(adventure_dict, filename_prefix="adventure_export"):
    """Save adventure as markdown file with timestamp"""
    md = render_markdown(adventure_dict)
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    path = os.path.join(EXPORT_DIR, f"{filename_prefix}-{ts}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(md)
    print(f"✅ Saved Markdown: {path}")
    return path

# Example usage (requires adventure_dict already defined):
# exported = save_markdown(adventure_dict)
# print(exported)

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-KEY'

In [34]:
# =========================
# OpenAI Expansion (Optional)
# =========================
# Requires: `pip install openai` (SDK v1.0+)
# Docs: platform.openai.com/docs/api-reference
# Set OPENAI_API_KEY env var before running.

import os, json
from datetime import datetime

try:
    from openai import OpenAI
    OPENAI_AVAILABLE = True
except ImportError:
    OPENAI_AVAILABLE = False
    print("OpenAI SDK not installed. Run: pip install openai")

# Use a valid model name - gpt-4o, gpt-4-turbo, or gpt-3.5-turbo
MODEL = os.environ.get("ADVENTURE_MODEL", "gpt-4o")
EXPORT_DIR = os.environ.get("ADVENTURE_EXPORT_DIR", "mnt/data/exports")
os.makedirs(EXPORT_DIR, exist_ok=True)

SYSTEM_PROMPT = (
    "You are a senior tabletop RPG designer.\n"
    "Generate a single self-contained Markdown adventure file for Dungeons & Dragons "
    "based on the provided JSON world/adventure object.\n"
    "Requirements:\n"
    "- Keep it CR-3 average unless the input says otherwise.\n"
    "- Include: Summary, 3 Hooks, Settlement, Dungeon, Monster Encounters (with stat blocks), "
    "Random Encounters (table), Treasure table (Tier-1 d6), Procedural tables for traps, relics, "
    "NPC dialogue, dynamic events, and Resolution paths.\n"
    "- Use clean headings and tables. Avoid purple prose. Keep it game-ready.\n"
    "- Do not include placeholder text; fill all required sections.\n"
)

USER_PROMPT_TEMPLATE = (
    "Input JSON (world/adventure):\n\n"
    "```json\n{json_blob}\n```\n\n"
    "Output:\nReturn only Markdown. No preface, no code fences.\n"
)

def call_openai_and_save(adventure_dict, filename_prefix="adventure_openai"):
    """
    Call OpenAI API to expand adventure JSON into styled Markdown.
    Returns the path to the saved file.
    """
    if not OPENAI_AVAILABLE:
        raise RuntimeError("OpenAI SDK not available. Install it first: pip install openai")
    
    api_key = os.environ.get("OPENAI_API_KEY")
    if not api_key:
        raise RuntimeError("OPENAI_API_KEY not set in environment.")
    
    client = OpenAI(api_key=api_key)
    json_blob = json.dumps(adventure_dict, ensure_ascii=False, indent=2)
    user_prompt = USER_PROMPT_TEMPLATE.format(json_blob=json_blob)

    print(f"Calling OpenAI API with model: {MODEL}...")
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": user_prompt},
            ],
            temperature=0.7,
            max_tokens=4000,
        )
        md_text = response.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"OpenAI API call failed: {e}")

    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    path = os.path.join(EXPORT_DIR, f"{filename_prefix}-{ts}.md")
    with open(path, "w", encoding="utf-8") as f:
        f.write(md_text)
    print(f"✅ Saved OpenAI Markdown: {path}")
    return path

# Example usage (requires adventure_dict already defined):
# openai_md = call_openai_and_save(adventure_dict)
# print(openai_md)

In [35]:
# =========================
# Schema Mapper (Optional)
# =========================
def map_to_standard_schema(src: dict) -> dict:
    """
    Map arbitrary world/adventure JSON to the standardized structure expected by render_markdown().
    Adjust the field accessors as needed for your generator.
    """
    adventure = {
        "title": src.get("quest", {}).get("output", "Untitled Adventure").split("\n")[0].strip() or "Untitled Adventure",
        "region": src.get("dungeon", {}).get("region") or src.get("quest", {}).get("region") or src.get("settlement", {}).get("region"),
        "theme":  src.get("dungeon", {}).get("theme") or src.get("quest", {}).get("hook", {}).get("type"),
        "quest_summary": {
            "type": src.get("quest", {}).get("hook", {}).get("type", ""),
            "description": f"{src.get('quest', {}).get('output','').strip()}",
            "complication": src.get("quest", {}).get("complication", ""),
            "reward": src.get("quest", {}).get("reward", ""),
        },
        "hooks": [
            {"title":"Hook A","description":"Lead from survivor testimony of a talking guardian."},
            {"title":"Hook B","description":"Rival faction offers payment to capture the monster."},
            {"title":"Hook C","description":"Local tomb-robbers vanish; one returns cursed."},
        ],
        "locations": {
            "settlement": {
                "name": f"{src.get('settlement', {}).get('region','Riftmarch')} City",
                "type": src.get("settlement", {}).get("type","settlement"),
                "population": src.get("settlement", {}).get("population"),
                "defense_rating": src.get("settlement", {}).get("defense_rating"),
                "market_size": src.get("settlement", {}).get("market_size"),
                "description": "A relic-trade hub built atop ancient ruins and fog-laden streets."
            },
            "dungeon": {
                "name": "The Howling Crypt",
                "depth": src.get("dungeon", {}).get("depth", 3),
                "cr_guess": src.get("dungeon", {}).get("cr_guess", 3),
                "description": "A cursed cliffside tomb where runes flicker with green fire."
            }
        },
        "monster_encounters": [
            {
                "name": "The Bound Guardian",
                "type": "Undead (Sapient)",
                "cr": 3,
                "description": "Once a paladin, now a skeletal warden bound by spectral chains.",
                "abilities": [
                    "Spectral Chains (Recharge 5–6): 10-ft AoE restrain, DC 13 STR save.",
                    "Voice of the Fallen: speaks Common; remembers oaths."
                ],
                "behavior": "Will parley if approached respectfully; desires release."
            },
            {
                "name": "Scarlet Covenant Agents",
                "type": "Humanoid Cultists",
                "cr": 2,
                "description": "Necromantic zealots seeking to enslave the guardian.",
                "abilities": ["Necrotic Bolt (1d8+2 necrotic, 60 ft.)","Fanatical Zeal"]
            }
        ],
        "random_encounters": [
            {"roll":"1-2","encounter":"Skeletal rat swarm gnawing glowing bone shards (CR 1/2)."},
            {"roll":"3-4","encounter":"Ghost lights lure explorers into rune traps."},
            {"roll":"5-6","encounter":"A lost spirit begs for help, leading to shadows."}
        ],
        "treasure": [
            {"roll":"1","treasure":"120 gp + bone-carved holy symbol (25 gp)"},
            {"roll":"2","treasure":"Scroll of Protection from Evil and Good + bloodstone (50 gp)"},
            {"roll":"3","treasure":"+1 Longsword (glows near undead)"},
            {"roll":"4","treasure":"3 potions of healing + 200 sp"},
            {"roll":"5","treasure":"Silver holy symbol (75 gp)"},
            {"roll":"6","treasure":"Relic: Helm fragment grants adv. on Insight vs undead (1/day)"}
        ],
        "procedural_traps": [
            {"roll":"1","trap":"Soul Snare Glyph","description":"DC 14 Dex; 2d10 necrotic."},
            {"roll":"2","trap":"Falling Sarcophagus Lid","description":"DC 15 Dex; 3d8 bludgeoning."},
            {"roll":"3","trap":"Cursed Whisper","description":"DC 12 Wis or 1d6 psychic; whispers 1h."},
            {"roll":"4","trap":"Spirit Ward","description":"Area glows; turn undead auto-triggers."},
            {"roll":"5","trap":"Rusted Spike Pit","description":"10-ft drop; 2d6 piercing; disease save DC 13."},
            {"roll":"6","trap":"Guardian’s Mark","description":"Reveals intruders to undead within 100 ft."}
        ],
        "procedural_relics": [
            {"roll":"1","relic":"Censer of Vigil","power":"+1 saves vs fear (1h) when lit."},
            {"roll":"2","relic":"Phantom Ring","power":"Pass through 5 ft. solid once/day."},
            {"roll":"3","relic":"Oathbreaker’s Pendant","power":"Negate one crit, then crumbles."},
            {"roll":"4","relic":"Tear of the Archon","power":"Blessed light; heals 1d4 when sanctified."},
            {"roll":"5","relic":"Sigil of Chains","power":"Immune to restrained once/long rest."},
            {"roll":"6","relic":"Crown of Riftmarch","power":"Awakens Ser Kael when placed on altar."}
        ],
        "procedural_dialogue": [
            {"roll":"1","npc":"Trader","line":"The beast spoke my name."},
            {"roll":"2","npc":"Covenant Emissary","line":"Redemption is a chain you cannot see."},
            {"roll":"3","npc":"Old Priest","line":"Kael’s vow binds his soul."},
            {"roll":"4","npc":"Robber","line":"The walls cried when torches died."},
            {"roll":"5","npc":"Guard Captain","line":"Take silver—and prayers."},
            {"roll":"6","npc":"Cult Adept","line":"His soul will power our rites."},
            {"roll":"7","npc":"Child","line":"The fog whispered your name."},
            {"roll":"8","npc":"Paladin","line":"Chains are promises made of iron."}
        ],
        "procedural_events": [
            {"roll":"1","event":"Spectral mist reveals hidden runes."},
            {"roll":"2","event":"Empty sarcophagus; fresh footprints."},
            {"roll":"3","event":"Covenant arrives mid-fight."},
            {"roll":"4","event":"Ghostly warning: 'They are here.'"},
            {"roll":"5","event":"Healing suppressed (30 ft., 1h)."},
            {"roll":"6","event":"Relic pulse purges undead; angers guardian."},
            {"roll":"7","event":"Buried spirits rise (ally/enemy)."},
            {"roll":"8","event":"Chains shatter—new curse or blessing."}
        ],
        "resolution": {
            "choices": [
                "Slay the guardian and claim the relics.",
                "Free the soul by invoking the true name at the altar.",
                "Strike a bargain with the Scarlet Covenant."
            ],
            "outcome":"Riftmarch’s fate hinges on the party’s choice."
        }
    }
    return {"adventure": adventure}


In [36]:
# =========================
# COMPLETE WORKFLOW
# =========================
# Run this cell to generate both local and OpenAI markdown exports

# Step 1: Map generator outputs to standard adventure schema
print("Step 1: Mapping outputs to adventure schema...")
adventure_dict = map_to_standard_schema(outputs)

# Step 2: Generate local markdown (no API required)
print("\nStep 2: Generating local markdown...")
local_md = save_markdown(adventure_dict, filename_prefix="adventure_local")

# Step 3: (Optional) Generate OpenAI-enhanced markdown
# Uncomment the lines below if you want to use OpenAI
# Make sure OPENAI_API_KEY is set in cell 7
print("\nStep 3: Generating OpenAI-enhanced markdown...")
openai_md = call_openai_and_save(adventure_dict, filename_prefix="adventure_openai")
print(f"OpenAI export: {openai_md}")

print("\n✅ All exports complete!")

Step 1: Mapping outputs to adventure schema...

Step 2: Generating local markdown...
✅ Saved Markdown: mnt/data/exports/adventure_local-20251111-153147.md

Step 3: Generating OpenAI-enhanced markdown...
Calling OpenAI API with model: gpt-4o...
✅ Saved OpenAI Markdown: mnt/data/exports/adventure_openai-20251111-153207.md
OpenAI export: mnt/data/exports/adventure_openai-20251111-153207.md

✅ All exports complete!


## 📖 Usage Guide

### Quick Start

```python
# 1. Map your generator output to standard schema (if needed)
standard = map_to_standard_schema(outputs)  # outputs from your world generator

# 2. Generate local markdown (no API required)
local_md = save_markdown(standard, filename_prefix="adventure_local")

# 3. (Optional) Generate OpenAI-enhanced markdown
# Requires: pip install openai
# Set environment variable: export OPENAI_API_KEY="sk-..."
openai_md = call_openai_and_save(standard, filename_prefix="adventure_openai")
```

### Environment Variables

- `OPENAI_API_KEY` - Your OpenAI API key (required for OpenAI expansion)
- `ADVENTURE_MODEL` - Model to use (default: `gpt-4o`, alternatives: `gpt-4-turbo`, `gpt-3.5-turbo`)
- `ADVENTURE_EXPORT_DIR` - Export directory (default: `/mnt/data/exports`)

### Notes

- **Local export** uses pure Python templating - no API calls, instant results
- **OpenAI export** calls the Chat Completions API for enhanced narrative and formatting
- Both exports follow the same schema structure for consistency
- The schema mapper helps convert your generator's output format to the standard structure